In [5]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [6]:
# Path ke folder train dan validation
train_dir = 'vehicle_dataset/train'
val_dir = 'vehicle_dataset/valid'

# Image parameters
img_height, img_width = 128, 128

In [7]:
def load_images_and_labels(image_dir, label_dir):
    images, labels = [], []
    for filename in tqdm(os.listdir(label_dir)):  # Iterasi file label
        if filename.endswith('.txt'):
            label_path = os.path.join(label_dir, filename)
            image_path = os.path.join(image_dir, filename.replace('.txt', '.jpg'))  # Asumsi gambar dan label memiliki nama yang sama

            # Baca file label
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    data = line.strip().split()
                    class_id = int(data[0])  # Class ID
                    x_center, y_center, width, height = map(float, data[1:])

                    # Baca gambar
                    image = cv2.imread(image_path)
                    if image is None:
                        continue

                    h, w, _ = image.shape
                    x_center, y_center = int(x_center * w), int(y_center * h)
                    box_width, box_height = int(width * w), int(height * h)

                    # Crop bounding box dari gambar
                    x1, y1 = x_center - box_width // 2, y_center - box_height // 2
                    x2, y2 = x_center + box_width // 2, y_center + box_height // 2
                    cropped_img = image[y1:y2, x1:x2]

                    # Resize ke ukuran input model
                    resized_img = cv2.resize(cropped_img, (img_height, img_width))

                    # Tambahkan ke dataset
                    images.append(resized_img)
                    labels.append(class_id)
    return np.array(images), np.array(labels)


In [11]:
# Load training data
train_images, train_labels = load_images_and_labels(
    os.path.join(train_dir, 'images'), os.path.join(train_dir, 'labels')
)

# Load validation data
val_images, val_labels = load_images_and_labels(
    os.path.join(val_dir, 'images'), os.path.join(val_dir, 'labels')
)

# Normalize images
train_images = train_images / 255.0
val_images = val_images / 255.0

# Convert labels to one-hot encoding
num_classes = len(set(train_labels))
train_labels = to_categorical(train_labels, num_classes)
val_labels = to_categorical(val_labels, num_classes)

100%|██████████| 900/900 [00:10<00:00, 87.37it/s] 


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10, batch_size=32)

# Save the model
model.save('vehicle_classifier.h5')

Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 15s 225ms/step - accuracy: 0.2467 - loss: 1.7472 - val_accuracy: 0.4734 - val_loss: 1.2217
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 12s 213ms/step - accuracy: 0.6212 - loss: 1.0935 - val_accuracy: 0.7150 - val_loss: 0.7795
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 13s 218ms/step - accuracy: 0.7474 - loss: 0.7289 - val_accuracy: 0.7500 - val_loss: 0.6951
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 13s 218ms/step - accuracy: 0.8083 - loss: 0.5616 - val_accuracy: 0.8007 - val_loss: 0.5565
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 240ms/step - accuracy: 0.8544 - loss: 0.4252 - val_accuracy: 0.8273 - val_loss: 0.4981
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 13s 226ms/step - accuracy: 0.8653 - loss: 0.3773 - val_accuracy: 0.8442 - val_loss: 0.4821
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 245ms/step - accuracy: 0.8742 - loss: 0.3413 - val_accuracy: 0.8128 - val_loss: 0.5992
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 14s 237ms/step - accuracy: 0.9032 - loss: 0.2647 - val_accu

In [15]:
# Load the model
model = tf.keras.models.load_model('vehicle_classifier.h5')

# Class labels
class_labels = {i: f'Class {i}' for i in range(num_classes)}

# Video processing
video_path = 'vehicle_dataset/vehicle-counting.mp4'
cap = cv2.VideoCapture(video_path)

vehicle_counts = {class_labels[i]: 0 for i in range(num_classes)}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    resized_frame = cv2.resize(frame, (img_height, img_width))
    input_frame = np.expand_dims(resized_frame / 255.0, axis=0)

    # Predict class
    predictions = model.predict(input_frame)
    predicted_class = class_labels[np.argmax(predictions)]

    # Count vehicle
    vehicle_counts[predicted_class] += 1

    # Display results
    cv2.putText(frame, f"Detected: {predicted_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Vehicle Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Print total counts
print("Vehicle Counts:")
for vehicle, count in vehicle_counts.items():
    print(f"{vehicle}: {count}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━